In [1]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


/home/zara/Onedrive/multifit_ner/env/lib/python3.8/site-packages/urllib3/util/retry.py:436: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  warnings.warn(
SDK version: 1.18.0
MLflow version: 1.12.1
zd-ner
scan-datasources-rg
southcentralus
d35ad7f3-a6be-400c-b1db-8b70322a01a0


In [2]:
from azureml.core import Experiment

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

experiment_name = "ner"
experiment = Experiment(ws, name=experiment_name)


/home/zara/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = 'conda.yaml')

# Specify a GPU base image
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

In [4]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      arguments=["--json_config", "training_config.json"],
                      compute_target="zd",
                      environment=pytorch_env)
run = experiment.submit(src)
RunDetails(run).show()

/home/zara/Onedrive/multifit_ner/env/lib/python3.8/site-packages/urllib3/util/retry.py:436: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  warnings.warn(


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…